# MUUFL数据集
来自仓库https://github.com/GatorSense/MUUFLGulfport
训练集和测试集的划分在MFT论文中采用了5%的训练集和95%的测试集。
>  The table represents class-specific land-cover types and the number of randomly selected (5%) training and the remaining (95%) test samples

本文件中采用sample_gt函数 同样随机选择5%的点来划分


In [1]:
import scipy.io as io
import numpy as np


In [2]:
label = io.loadmat('muufl_gulfport_campus_1_hsi_220_label.mat')
print(label)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Nov 01 17:19:22 2017', '__version__': '1.0', '__globals__': [], 'hsi': array([[(array([[(array([[array(['{Empirical Line Result [Thu Dec 09 10:53:58 2010]}'], dtype='<U50'),
                        array(['Subimage of 1127_with_1135_CFF reflectance image over University of Southern Mississipi Gulf Park campus and emplaced targets, 1m resolution (downsampled from .5m) from 3500ft'],
                              dtype='<U175')                                                                                                                                                                      ]],
                      dtype=object), array([[220]], dtype=uint8), array([[325]], dtype=uint16), array([[64]], dtype=uint8), array(['HSI'], dtype='<U3'), array([[(array(['UTM'], dtype='<U3'), array([[1]], dtype=uint8), array([[1]], dtype=uint8), array([[16]], dtype=uint8), array(['North'], dtype='<U5'), array(['WGS-84'], dtype=

In [3]:
label = label['hsi'][0]

In [4]:
label

array([(array([[(array([[array(['{Empirical Line Result [Thu Dec 09 10:53:58 2010]}'], dtype='<U50'),
                       array(['Subimage of 1127_with_1135_CFF reflectance image over University of Southern Mississipi Gulf Park campus and emplaced targets, 1m resolution (downsampled from .5m) from 3500ft'],
                             dtype='<U175')                                                                                                                                                                      ]],
                     dtype=object), array([[220]], dtype=uint8), array([[325]], dtype=uint16), array([[64]], dtype=uint8), array(['HSI'], dtype='<U3'), array([[(array(['UTM'], dtype='<U3'), array([[1]], dtype=uint8), array([[1]], dtype=uint8), array([[16]], dtype=uint8), array(['North'], dtype='<U5'), array(['WGS-84'], dtype='<U6'), array(['Meters'], dtype='<U6'))]],
                     dtype=[('projection', 'O'), ('dx', 'O'), ('dy', 'O'), ('zone', 'O'), ('hemi', 'O'), 

In [5]:
label['sceneLabels'][0][0]

array([(array([[array(['trees'], dtype='<U5'),
               array(['grass_pure'], dtype='<U10'),
               array(['grass_groundsurface'], dtype='<U19'),
               array(['dirt_and_sand'], dtype='<U13'),
               array(['road_materials'], dtype='<U14'),
               array(['water'], dtype='<U5'),
               array(['shadow_building'], dtype='<U15'),
               array(['buildings'], dtype='<U9'),
               array(['sidewalk'], dtype='<U8'),
               array(['yellowcurb'], dtype='<U10'),
               array(['cloth_panels'], dtype='<U12')]], dtype=object), array([[array([[34],
                      [35],
                      [31],
                      ...,
                      [12],
                      [13],
                      [14]], dtype=uint16), array([[50],
                                                   [50],
                                                   [51],
                                                   ...,
                 

In [6]:
label['sceneLabels'][0][0]['Materials_Type'][0][0]

array([array(['trees'], dtype='<U5'), array(['grass_pure'], dtype='<U10'),
       array(['grass_groundsurface'], dtype='<U19'),
       array(['dirt_and_sand'], dtype='<U13'),
       array(['road_materials'], dtype='<U14'),
       array(['water'], dtype='<U5'),
       array(['shadow_building'], dtype='<U15'),
       array(['buildings'], dtype='<U9'),
       array(['sidewalk'], dtype='<U8'),
       array(['yellowcurb'], dtype='<U10'),
       array(['cloth_panels'], dtype='<U12')], dtype=object)

In [7]:
x = label['sceneLabels'][0][0]['Materials_rowIndices'][0][0]
x

array([array([[34],
              [35],
              [31],
              ...,
              [12],
              [13],
              [14]], dtype=uint16), array([[50],
                                           [50],
                                           [51],
                                           ...,
                                           [30],
                                           [31],
                                           [32]], dtype=uint16),
       array([[ 76],
              [ 77],
              [ 78],
              ...,
              [207],
              [208],
              [209]], dtype=uint16), array([[  2],
                                            [  1],
                                            [  2],
                                            ...,
                                            [323],
                                            [324],
                                            [325]], dtype=uint16),
       array([[157],
       

In [8]:
y = label['sceneLabels'][0][0]['Materials_colIndices'][0][0]
y

array([array([[  2],
              [  2],
              [  3],
              ...,
              [217],
              [217],
              [217]], dtype=uint8), array([[102],
                                           [103],
                                           [103],
                                           ...,
                                           [175],
                                           [175],
                                           [175]], dtype=uint8),
       array([[ 74],
              [ 74],
              [ 74],
              ...,
              [149],
              [149],
              [149]], dtype=uint8), array([[164],
                                           [165],
                                           [165],
                                           ...,
                                           [220],
                                           [220],
                                           [220]], dtype=uint8),
       array([[  3],
     

In [9]:
gt = np.zeros((325,220))
for i in range( len(x)):
    print(i,len(x[i]))
    for j in range(len(x[i])):
        xx = x[i][j][0]-1
        yy = y[i][j][0]-1
        # print(xx,yy)
        gt[xx][yy]=i+1

gt



0 23885
1 4275
2 6913
3 1842
4 6940
5 466
6 2235
7 6240
8 1438
9 187
10 271


array([[8., 8., 8., ..., 0., 0., 3.],
       [8., 8., 8., ..., 0., 3., 3.],
       [8., 8., 8., ..., 3., 3., 3.],
       ...,
       [8., 8., 8., ..., 4., 4., 4.],
       [8., 0., 8., ..., 4., 4., 4.],
       [0., 0., 0., ..., 4., 4., 4.]])

In [10]:
import sklearn
import sklearn.model_selection
def sample_gt(gt, train_size, mode='random'):
    if train_size==1:
        return gt
    """Extract a fixed percentage of samples from an array of labels.

    Args:
        gt: a 2D array of int labels
        percentage: [0, 1] float
    Returns:
        train_gt, test_gt: 2D arrays of int labels

    """
    indices = np.nonzero(gt)
    X = list(zip(*indices)) # x,y features
    y = gt[indices].ravel() # classes
    train_gt = np.zeros_like(gt)
    test_gt = np.zeros_like(gt)
    if train_size > 1:
       train_size = int(train_size)

    if mode == 'random':
       train_indices, test_indices = sklearn.model_selection.train_test_split(X, train_size=train_size, stratify=y)
       train_indices = tuple(list(t) for t in zip(*train_indices))
       test_indices = tuple(list(t) for t in zip(*test_indices))
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]
    elif mode == 'fixed':
       print("Sampling {} with train size = {}".format(mode, train_size))
       train_indices, test_indices = [], []
       for c in np.unique(gt):
           if c == 0:
              continue
           indices = np.nonzero(gt == c)
           X = list(zip(*indices)) # x,y features

           train, test = sklearn.model_selection.train_test_split(X, train_size=train_size)
           train_indices += train
           test_indices += test
       train_indices = [list(t) for t in zip(*train_indices)]
       test_indices = [list(t) for t in zip(*test_indices)]
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]

    elif mode == 'disjoint':
        train_gt = np.copy(gt)
        test_gt = np.copy(gt)
        for c in np.unique(gt):
            mask = gt == c
            for x in range(gt.shape[0]):
                first_half_count = np.count_nonzero(mask[:x, :])
                second_half_count = np.count_nonzero(mask[x:, :])
                try:
                    ratio = first_half_count / (first_half_count + second_half_count)
                    if ratio > 0.9 * train_size:
                        break
                except ZeroDivisionError:
                    continue
            mask[:x, :] = 0
            train_gt[mask] = 0

        test_gt[train_gt > 0] = 0
    else:
        raise ValueError("{} sampling is not implemented yet.".format(mode))
    return train_gt,test_gt
def choose_true_point(true_data, num_classes):
    number_true = []
    pos_true = {}
    for i in range(num_classes + 1):
        each_class = np.argwhere(true_data == i)
        number_true.append(each_class.shape[0])
        pos_true[i] = each_class

    total_pos_true = pos_true[0]
    for i in range(1, num_classes + 1):
        total_pos_true = np.r_[total_pos_true, pos_true[i]]
    total_pos_true = total_pos_true.astype(int)
    return total_pos_true, number_true


def choose_img_point(height, width):
    total_pos_true = np.array([[i, j] for i in range(height) for j in range(width)])
    return total_pos_true


# 1
def chooose_point(test_data, num_classes):
    number_test = []
    pos_test = {}

    for i in range(num_classes):
        each_class = np.argwhere(test_data == (i + 1))
        number_test.append(each_class.shape[0])
        pos_test[i] = each_class

    total_pos_test = pos_test[0]
    for i in range(1, num_classes):
        total_pos_test = np.r_[total_pos_test, pos_test[i]]  # (9671,2)
    total_pos_test = total_pos_test.astype(int)
    return total_pos_test, number_test


def mirror_hsi( input_normalize, patch=5):
    height, width, band = input_normalize.shape
    padding = patch // 2
    mirror_hsi = np.zeros((height + 2 * padding, width + 2 * padding, band), dtype=float)  # padding后的图 上下左右各加padding

    mirror_hsi[padding:(padding + height), padding:(padding + width), :] = input_normalize  # 中间用原图初始化

    for i in range(padding):
        mirror_hsi[padding:(height + padding), i, :] = input_normalize[:, padding - i - 1, :]

    for i in range(padding):
        mirror_hsi[padding:(height + padding), width + padding + i, :] = input_normalize[:, width - 1 - i, :]

    for i in range(padding):
        mirror_hsi[i, :, :] = mirror_hsi[padding * 2 - i - 1, :, :]

    for i in range(padding):
        mirror_hsi[height + padding + i, :, :] = mirror_hsi[height + padding - 1 - i, :, :]

    print("**************************************************")
    print("patch is : {}".format(patch))
    print("mirror_image shape : [{0},{1},{2}]".format(mirror_hsi.shape[0], mirror_hsi.shape[1], mirror_hsi.shape[2]))
    print("**************************************************")
    return mirror_hsi


def gain_neighborhood_pixel(mirror_image, point, i, patch=5):
    x = point[i, 0]
    y = point[i, 1]
    temp_image = mirror_image[x:(x + patch), y:(y + patch), :]
    return temp_image




def get_data(mirror_image, test_point, patch=5):
    height, width, band = mirror_image.shape
    x_test = np.zeros((test_point.shape[0], patch, patch, band), dtype=float)

    for j in range(test_point.shape[0]):
        x_test[j, :, :, :] = gain_neighborhood_pixel(mirror_image, test_point, j, patch)
    print("x_test  shape = {}, type = {}".format(x_test.shape, x_test.dtype))
    print("**************************************************")

    return x_test


def get_label(number_test, num_classes):
    y_test = []
    for i in range(num_classes):
        for k in range(number_test[i]):
            y_test.append(i+1)

    y_test = np.array(y_test)
    print("y_test: shape = {} ,type = {}".format(y_test.shape, y_test.dtype))
    print("**************************************************")
    return y_test

def norm(data):
    input_normalize = np.zeros(data.shape)

    for i in range(data.shape[2]):
        input_max = np.max(data[:, :, i])
        input_min = np.min(data[:, :, i])
        input_normalize[:, :, i] = (data[:, :, i] - input_min) / (input_max - input_min)
    return input_normalize

划分出5%的训练集和95%的测试集

In [11]:
train_gt,test_gt = sample_gt(gt,0.05)

In [12]:
train_gt

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 4., 0., 0.]])

In [13]:
type(label)

numpy.ndarray

In [14]:
label['info'][0][0]

array([(array([[array(['{Empirical Line Result [Thu Dec 09 10:53:58 2010]}'], dtype='<U50'),
               array(['Subimage of 1127_with_1135_CFF reflectance image over University of Southern Mississipi Gulf Park campus and emplaced targets, 1m resolution (downsampled from .5m) from 3500ft'],
                     dtype='<U175')                                                                                                                                                                      ]],
             dtype=object), array([[220]], dtype=uint8), array([[325]], dtype=uint16), array([[64]], dtype=uint8), array(['HSI'], dtype='<U3'), array([[(array(['UTM'], dtype='<U3'), array([[1]], dtype=uint8), array([[1]], dtype=uint8), array([[16]], dtype=uint8), array(['North'], dtype='<U5'), array(['WGS-84'], dtype='<U6'), array(['Meters'], dtype='<U6'))]],
             dtype=[('projection', 'O'), ('dx', 'O'), ('dy', 'O'), ('zone', 'O'), ('hemi', 'O'), ('datum', 'O'), ('units', 'O')]), array([

In [15]:
label['Data'][0]

array([[[0.128792  , 0.13693085, 0.13503247, ..., 0.30102918,
         0.30894166, 0.30242357],
        [0.12960756, 0.13706912, 0.13715482, ..., 0.29628858,
         0.29806232, 0.30666316],
        [0.14245248, 0.1429686 , 0.14121084, ..., 0.28560448,
         0.30321568, 0.33222923],
        ...,
        [0.00141325, 0.0144708 , 0.01849237, ..., 0.19142893,
         0.2069558 , 0.22527578],
        [0.0051342 , 0.02433398, 0.01028598, ..., 0.33513334,
         0.3594574 , 0.36749494],
        [0.01823398, 0.01594567, 0.01703031, ..., 0.35338825,
         0.3593938 , 0.37558872]],

       [[0.1264473 , 0.13462637, 0.12918425, ..., 0.28737336,
         0.29418138, 0.31552777],
        [0.13358337, 0.13789874, 0.13234417, ..., 0.2967131 ,
         0.30499712, 0.3247778 ],
        [0.12670216, 0.13702303, 0.1376736 , ..., 0.28503844,
         0.30925977, 0.3136649 ],
        ...,
        [0.02531907, 0.02963428, 0.02542535, ..., 0.3192841 ,
         0.3521409 , 0.360172  ],
        [0.0

In [16]:
label['Data'][0].shape

(325, 220, 64)

In [17]:
hsi = label['Data'][0]

In [18]:
label['Lidar'][0]

array([[array([[(array([[294566.264, 294567.264, 294568.264, 294569.264, 294570.264,
                        294571.264, 294572.264, 294573.264, 294574.264, 294575.264,
                        294576.264, 294577.264, 294578.264, 294579.264, 294580.264,
                        294581.264, 294582.264, 294583.264, 294584.264, 294585.264,
                        294586.264, 294587.264, 294588.264, 294589.264, 294590.264,
                        294591.264, 294592.264, 294593.264, 294594.264, 294595.264,
                        294596.264, 294597.264, 294598.264, 294599.264, 294600.264,
                        294601.264, 294602.264, 294603.264, 294604.264, 294605.264,
                        294606.264, 294607.264, 294608.264, 294609.264, 294610.264,
                        294611.264, 294612.264, 294613.264, 294614.264, 294615.264,
                        294616.264, 294617.264, 294618.264, 294619.264, 294620.264,
                        294621.264, 294622.264, 294623.264, 294624.264, 294

In [19]:
label['Lidar'][0].shape

(1, 2)

In [20]:
label['Lidar'][0]

array([[array([[(array([[294566.264, 294567.264, 294568.264, 294569.264, 294570.264,
                        294571.264, 294572.264, 294573.264, 294574.264, 294575.264,
                        294576.264, 294577.264, 294578.264, 294579.264, 294580.264,
                        294581.264, 294582.264, 294583.264, 294584.264, 294585.264,
                        294586.264, 294587.264, 294588.264, 294589.264, 294590.264,
                        294591.264, 294592.264, 294593.264, 294594.264, 294595.264,
                        294596.264, 294597.264, 294598.264, 294599.264, 294600.264,
                        294601.264, 294602.264, 294603.264, 294604.264, 294605.264,
                        294606.264, 294607.264, 294608.264, 294609.264, 294610.264,
                        294611.264, 294612.264, 294613.264, 294614.264, 294615.264,
                        294616.264, 294617.264, 294618.264, 294619.264, 294620.264,
                        294621.264, 294622.264, 294623.264, 294624.264, 294

In [21]:
label['Lidar'][0][0][0][0]

array([(array([[294566.264, 294567.264, 294568.264, 294569.264, 294570.264,
               294571.264, 294572.264, 294573.264, 294574.264, 294575.264,
               294576.264, 294577.264, 294578.264, 294579.264, 294580.264,
               294581.264, 294582.264, 294583.264, 294584.264, 294585.264,
               294586.264, 294587.264, 294588.264, 294589.264, 294590.264,
               294591.264, 294592.264, 294593.264, 294594.264, 294595.264,
               294596.264, 294597.264, 294598.264, 294599.264, 294600.264,
               294601.264, 294602.264, 294603.264, 294604.264, 294605.264,
               294606.264, 294607.264, 294608.264, 294609.264, 294610.264,
               294611.264, 294612.264, 294613.264, 294614.264, 294615.264,
               294616.264, 294617.264, 294618.264, 294619.264, 294620.264,
               294621.264, 294622.264, 294623.264, 294624.264, 294625.264,
               294626.264, 294627.264, 294628.264, 294629.264, 294630.264,
               294631.26

In [22]:
label['Lidar'][0][0][0][0]['z'][0].shape

(325, 220, 2)

In [23]:
lidar = label['Lidar'][0][0][0][0]['z'][0]

In [24]:
num_classes = 11

#
total_pos_test, number_test = chooose_point(test_gt, num_classes)
total_pos_train, number_train = chooose_point(train_gt, num_classes)


In [25]:
len(total_pos_train)

2684

In [26]:
hsi = norm(hsi)
io.savemat(r'HSI_norm.mat',{'Data':hsi})
hsi = mirror_hsi(hsi,11)

HSI_Te = get_data(hsi, total_pos_test,patch=11,)
io.savemat("HSI_Te.mat",{"Data":HSI_Te})
del HSI_Te


HSI_Tr = get_data(hsi, total_pos_train,patch=11,)
io.savemat("HSI_Tr.mat",{"Data":HSI_Tr})
del HSI_Tr
del  hsi


**************************************************
patch is : 11
mirror_image shape : [335,230,64]
**************************************************
x_test  shape = (51003, 11, 11, 64), type = float64
**************************************************
x_test  shape = (2684, 11, 11, 64), type = float64
**************************************************


In [27]:
lidar = norm(lidar)
io.savemat(r'LIDAR_norm.mat',{'Data':lidar})
lidar = mirror_hsi(lidar,11)

LIDAR_Te = get_data(lidar, total_pos_test,patch=11,)
io.savemat("LIDAR_Te.mat",{"Data":LIDAR_Te})
del LIDAR_Te


LIDAR_Tr = get_data(lidar, total_pos_train,patch=11,)
io.savemat("LIDAR_Tr.mat",{"Data":LIDAR_Tr})

**************************************************
patch is : 11
mirror_image shape : [335,230,2]
**************************************************
x_test  shape = (51003, 11, 11, 2), type = float64
**************************************************
x_test  shape = (2684, 11, 11, 2), type = float64
**************************************************


In [28]:
TeLabel = get_label(number_test, num_classes)
TrLabel = get_label(number_train, num_classes)
io.savemat("TeLabel.mat",{"Data":TeLabel})
io.savemat("TrLabel.mat",{"Data":TrLabel})

y_test: shape = (51003,) ,type = int64
**************************************************
y_test: shape = (2684,) ,type = int64
**************************************************


In [29]:
TeLabel

array([ 1,  1,  1, ..., 11, 11, 11])

In [30]:
TrLabel

array([ 1,  1,  1, ..., 11, 11, 11])